# 🎬 영화 추천시스템 만들기
**"내가 좋아하는 영화를 자동으로 찾아주는 알고리즘 제작 프로젝트"**

#### 목차
1. 데이터
 - 데이터 불러오기
 - 데이터 전처리하기
 - 데이터 분석하기
 
 
2. 모델
 - 모델 만들기
 - 모델 실행하기
 
 
3. 회고
 - 요약
 - 결론

---

## 1. 데이터

### 데이터 불러오기

In [1]:
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/aiffel_exploration/E09/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


> 유저 데이터가 깔끔하게 정리되어 있는 파일이다.

### 데이터 전처리하기

In [2]:
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


> 3점 미만 데이터는 삭제(유저가 선호하지 않는다고 판단)

In [3]:
ratings.rename(columns={'rating':'count'}, inplace=True)
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

> 별점을 시청횟수로 해석

In [4]:
movie_file_path=os.getenv('HOME') + '/aiffel/aiffel_exploration/E09/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


> 영화 데이터가 잘 정리되어 있다.

In [5]:
movie_data = pd.merge(ratings, movies, how = 'left', on = 'movie_id')
movie_data.head()

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


유저 데이터와 영화 데이터를 moive_id를 기준으로 합친다.

### 데이터 분석하기

#### ratings에 있는 유니크한 영화 개수

In [6]:
ratings['movie_id'].nunique()

3628

#### rating에 있는 유니크한 사용자 수

In [7]:
ratings['user_id'].nunique()

6039

#### 가장 인기 있는 영화 30개(인기순)

In [8]:
movie_count = movie_data.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

> 스타워즈 시리즈가 상위권에 있다. 필자 또한 매우 좋아하는 영화이다.

---

## 2. 모델

### 모델 만들기

#### 내가 선호하는 영화 5가지 rating에 추가

In [9]:
# 모델링에 사용하지 않는 데이터 제거
movie_data = movie_data.drop(columns=['timestamp', 'genre','movie_id'])                                     

In [10]:
# 제목 칼럼
my_favorite = ['Star Wars: Episode VI - Return of the Jedi (1983)' , 'Terminator 2: Judgment Day (1991)', 'Back to the Future (1985)' ,
               'Matrix, The (1999)' ,'Jurassic Park (1993)']

# 표 만들기
my_data = pd.DataFrame({'user_id' : ['wbj1209']*5, 'title' : my_favorite, 'count' : [5]*5})

# my_data를 데이터를 기존 데이터에 추가
if not movie_data.isin({'user_id' : ['wbj1209']})['user_id'].any() :
    movie_data = movie_data.append(my_data)
    
movie_data.tail(10)

,user_id,count,title
836473,6040,3,Platoon (1986)
836474,6040,5,"Crying Game, The (1992)"
836475,6040,5,Welcome to the Dollhouse (1995)
836476,6040,4,Sophie's Choice (1982)
836477,6040,4,E.T. the Extra-Terrestrial (1982)
0,wbj1209,5,Star Wars: Episode VI - Return of the Jedi (1983)
1,wbj1209,5,Terminator 2: Judgment Day (1991)
2,wbj1209,5,Back to the Future (1985)
3,wbj1209,5,"Matrix, The (1999)"
4,wbj1209,5,Jurassic Park (1993)


#### CSR matrix

In [11]:
# 고유한 유저, 타이틀
user_unique = movie_data['user_id'].unique()
title_unique = movie_data['title'].unique()

# 유저,타이틀 indexing
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

# 인덱싱이 잘 되었는지 확인
print(user_to_idx['wbj1209']) # 6040명의 유저 중 마지막으로 추가된 유저이니 6039이 나와야 함

6039


In [12]:
# 데이터 컬럼 값을 indexing 값으로 변경

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = movie_data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(movie_data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    movie_data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')
    
# temp_title_data을 통해 title 컬럼도 동일한 방식으로 인덱싱 해줍니다. 
temp_title_data = movie_data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(movie_data):
    print('title column indexing OK!!')
    movie_data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

movie_data

user_id column indexing OK!!
title column indexing OK!!


,user_id,count,title
0,0,5,0
1,0,3,1
2,0,3,2
3,0,4,3
4,0,5,4
...,...,...,...
0,6039,5,64
1,6039,5,92
2,6039,5,22
3,6039,5,124


In [13]:
from scipy.sparse import csr_matrix

num_user = movie_data['user_id'].nunique()
num_movie = movie_data['title'].nunique()

csr_data = csr_matrix((movie_data['count'], (movie_data.user_id, movie_data.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

#### MF 모델 훈련

In [14]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [15]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [16]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [17]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

### 모델 실행하기

#### 선호/비선호 영화와의 벡터값 차이

In [18]:
# 모델의 예측(내가 선호하는 영화)
wbj1209, matrix = user_to_idx['wbj1209'], title_to_idx['Matrix, The (1999)']
wbj1209_vector, matrix_vector = als_model.user_factors[wbj1209], als_model.item_factors[matrix]
np.dot(wbj1209_vector, matrix_vector)

0.6443497

In [19]:
# 모델의 예측(내가 선호하지 않는 영화)
fugitive = title_to_idx['Fugitive, The (1993)']
fugitive_vector = als_model.item_factors[fugitive]
np.dot(wbj1209_vector, fugitive_vector)

0.24380147

#### 내가 좋아하는 영화와 비슷한 영화 추천

In [20]:
# index로부터 title을 얻는 dict를 생성합니다.
idx_to_title = {v:k for k,v in title_to_idx.items()}

def get_similar_movie(movie_name: str):
    movie_id = title_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_title[i[0]] for i in similar_movie]
    return similar_movie

In [21]:
get_similar_movie('Matrix, The (1999)')

['Matrix, The (1999)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Fugitive, The (1993)',
 'Terminator, The (1984)',
 'Fifth Element, The (1997)',
 'Face/Off (1997)',
 'Jurassic Park (1993)',
 'Men in Black (1997)',
 'Star Wars: Episode IV - A New Hope (1977)']

> 꽤 잘 예측하는 것으로 보인다.

#### 내가 좋아할 만한 영화들을 추천

In [22]:
user = user_to_idx['wbj1209']

# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)

[idx_to_title[i[0]] for i in movie_recommended]

['Star Wars: Episode IV - A New Hope (1977)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Men in Black (1997)',
 'Total Recall (1990)',
 'Terminator, The (1984)',
 'Braveheart (1995)',
 'E.T. the Extra-Terrestrial (1982)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Raiders of the Lost Ark (1981)',
 'Saving Private Ryan (1998)',
 'Forrest Gump (1994)',
 'Sixth Sense, The (1999)',
 'Fugitive, The (1993)',
 'American Beauty (1999)',
 'Alien (1979)',
 'X-Men (2000)',
 'Aliens (1986)',
 'Airplane! (1980)',
 'Lost World: Jurassic Park, The (1997)',
 'Princess Bride, The (1987)']

> 이 또한 꽤 잘 예측하는 것으로 보인다.

---

## 3. 회고

**CSR matrix**
- 데이터량이 방대함에 비해 개개인이 접하는 정보는 적음
- 대부분의 공간이 0으로 채워짐. 이런 행렬을 Sparse Matrix
- 메모리 낭비를 최소화하기 위해 유저가 들어본 아티스트의 정보만을 저장하면서 전체 행렬 형태를 유추할 수 있는 데이터 구조


**MF 모델**
- Matrix Factorization 모델
- implicit 패키지는 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습 가능
- AlternatingLeastSquares 방식


**추천 결과**
실제로 추천해준 영화 목록의 경우 꽤나 잘 추천해줌을 확인하였다. 물론 좋아하는 장르가 뚜렷해서 그럴 수 있다. 벡터 사이의 거리에 대한 기준점이 애매하다. 추천시스템에 대한 원리를 꼼꼼히 살펴볼 예정이다.